In [1]:
import os

os.chdir("..")

current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn


In [2]:
# Import necessary modules and functions
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.neural_nets import create_neural_1, create_neural_3, create_neural_6, parameter_tuning_nn, runs_nn
from data.data_generator import get_data, preprocess

ImportError: cannot import name 'parameter_tuning' from 'src.neural_nets' (c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\neural_nets.py)

In [3]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20, training=True)
val_data = preprocess(x_val, y_val, batch_size=20, training=False)

units = [5,10,25,50,75]
best_hp = parameter_tuning_nn(
    create_neural_1,
    train_data = train_data,
    test_data = val_data,
    input_dim = input_dim,
    units = units
)

print(best_hp)

c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Unit: 5, MSE: 2.071204423904419, MAE: 1.2795650959014893
Unit: 10, MSE: 4.51339054107666, MAE: 2.015465259552002
Unit: 25, MSE: 0.1647074967622757, MAE: 0.2992742657661438
Unit: 50, MSE: 0.19104525446891785, MAE: 0.3664892613887787
Unit: 75, MSE: 0.16043494641780853, MAE: 0.31593984365463257
{'best_hp': 75, 'mse': 0.16043494641780853}


In [5]:
unit = best_hp['best_hp']
mses_neural_1, maes_neural_1 = runs_nn(
    create_neural_1,
    input_dim=input_dim,
    regression_func=regression_func,
    units=unit,
    epochs=100,
    batch_size=20,
    )

c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\max-s\AppData\Local\Temp\ipykernel_3952\2755177504.py", line 2, in <module>
    mses_neural_1, maes_neural_1 = runs_nn(
                                   ^^^^^^^^
  File "c:\Users\max-s\Desktop\Uni\Master\Masterthesis\Code\git\overparametrized_dnn\src\neural_nets.py", line 155, in runs_nn
TypeError: train_and_evaluate_nn() got an unexpected keyword argument 'batch_size'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\max-s\anaconda3\envs\py4ds\Lib\site-packages\IPytho